# Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import fitbit
import gather_keys_oauth2 as Oauth2
import MyFitnessReport_Keys as keys
import pandas as pd 
import datetime

# TODO : Get the keys from text file and remember to GitIgnore the textfile!
CLIENT_ID = keys.client_id
CLIENT_SECRET = keys.client_secret

# API Authorization 

In [2]:
server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[04/Mar/2020:23:11:39] ENGINE Listening for SIGTERM.
[04/Mar/2020:23:11:39] ENGINE Listening for SIGHUP.
[04/Mar/2020:23:11:39] ENGINE Listening for SIGUSR1.
[04/Mar/2020:23:11:39] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[04/Mar/2020:23:11:39] ENGINE Started monitor thread '_TimeoutMonitor'.
[04/Mar/2020:23:11:39] ENGINE Started monitor thread 'Autoreloader'.
[04/Mar/2020:23:11:39] ENGINE Serving on http://127.0.0.1:8080
[04/Mar/2020:23:11:39] ENGINE Bus STARTED


127.0.0.1 - - [04/Mar/2020:23:11:56] "GET /?code=c4d97971973e5f58a41c9fe9271568ab3e3af9b3&state=6tlCMDp5K1KBo5mD0e3aKNC5QTbh1D HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"


[04/Mar/2020:23:11:57] ENGINE Bus STOPPING
[04/Mar/2020:23:12:02] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[04/Mar/2020:23:12:02] ENGINE Stopped thread 'Autoreloader'.
[04/Mar/2020:23:12:02] ENGINE Stopped thread '_TimeoutMonitor'.
[04/Mar/2020:23:12:02] ENGINE Bus STOPPED
[04/Mar/2020:23:12:02] ENGINE Bus EXITING
[04/Mar/2020:23:12:02] ENGINE Bus EXITED
[04/Mar/2020:23:12:02] ENGINE Waiting for child threads to terminate...


# Get One Day of Data (Activities / Heart rate)

## Today's Heart Data 

In [4]:
#today = pd.datetime.today().date() # - datetime.timedelta(days=1)
#For consistency, today will be the temporary test day
today = pd.datetime(year = 2020, month = 2, day = 28)

In [5]:
todayHeartData = auth2_client.intraday_time_series('activities/heart', base_date=today, detail_level='1sec')
todayHeartData
# putting json data of heart activity into a dataframe
heartDf = pd.DataFrame(todayHeartData['activities-heart-intraday']['dataset'])
heartDf.head()

,time,value
0,09:09:09,70
1,09:09:24,69
2,09:09:29,74
3,09:09:39,75
4,09:09:44,76


In [7]:
# The first part gets a date in a string format of YYYY-MM-DD
filename = todayHeartData['activities-heart'][0]['dateTime'] +'_intradata'

# Export file to csv
heartDf.to_csv('res/'+ filename + '.csv', index = False)

## Heart Data (Get data for the month)

In [12]:
startTime = pd.datetime(year = 2020, month = 1, day = 28)
endTime = today 
date_list = []
df_list = []
final_df_list = []
allDates = pd.date_range(start=startTime, end = endTime)

for oneDate in allDates:
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    oneDayData = auth2_client.intraday_time_series('activities/heart', base_date=oneDate, detail_level='1sec')
    
    df = pd.DataFrame(oneDayData['activities-heart-intraday']['dataset'])
    date_list.append(oneDate)
    df_list.append(df)

for date, df in zip(date_list, df_list):
    if len(df) == 0:
        continue
    df.loc[:, 'date'] = pd.to_datetime(date)
    final_df_list.append(df)
final_df = pd.concat(final_df_list, axis = 0)


In [13]:
## Making of the data have more detailed timestamp (day and hour instead of day)
hoursDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.hour.apply(lambda x: datetime.timedelta(hours = x))
minutesDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.minute.apply(lambda x: datetime.timedelta(minutes = x))
secondsDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.second.apply(lambda x: datetime.timedelta(seconds = x))

# Getting the date to also have the time of the day
final_df['date'] = final_df['date'] + hoursDelta + minutesDelta + secondsDelta

In [14]:
final_df.head()

,time,value,date
0,00:00:00,73,2020-01-28 00:00:00
1,00:00:05,72,2020-01-28 00:00:05
2,00:00:10,69,2020-01-28 00:00:10
3,00:00:15,67,2020-01-28 00:00:15
4,00:00:20,67,2020-01-28 00:00:20


In [15]:
filename = 'all_intradata'
final_df.to_csv('res/' + filename + '.csv', index = False)